## KDD 1. 신용카드 사기 검출 실습

In [1]:
import pandas as pd
from pandas import Series, DataFrame

In [4]:
card_df = pd.read_csv('data/신용카드사기검출/creditcard.csv')

OSError: Initializing from file failed

In [ ]:
# 2. 데이터 탐색
card_df.info()
# 284,807 건의 데이터
# 총 31개의 컬럼, 모두 숫자형 타입 --> 인코딩 필요 X
# 결측치 없음

In [ ]:
# 클래스의 비율 확인
card_df.value_counts() / len(card_df) * 100
# 0 : 일반적인 카드 사용
# 1 : 카드 도용(사기)

In [ ]:
# 전처리 v1
# Time 컬럼 삭제
def get_preprocessed_df(df) :
    df2 = df.copy()
    # df2에 전처리 수행
    df2.drop('Time', axs = 1, inplace = True)
    return df2

In [ ]:
# 학습/검증용 데이터 분리
from sklearn.model_selection as ms

def get_train_test_datasets(df) :
    # 1) 전처리 수행
    df2 = get_preprocessed_df(df)
    # 2) 특성과 레이블(타겟 클래스) 분리
    X_features = df2.iloc[:, :-1]
    y_target = df2.Class  # df.iloc[:, -1]
    # 3) 학습/검증용 데이터 분리
    X_train, X_test, y_train, y_test = ms.train_test_split(X_features, y_target,
                                                          test_size = 0.2, 
                                                          random_state = 100)
    
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = get_train_test_datasets(card_df)

In [5]:
# 모델 선택 및 학습

In [ ]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 3000)
lr_clf.fit(X_train, y_train)

In [ ]:
get_clf_eval(y_test, lr_clf.predict(X_test))

In [ ]:
# count(negative) / count(positive)
weight = y_train.value_counts()[0] / y_train.value_counts()[1]

In [ ]:
# XGBoost로 학습/평가
from xgboost import XGBClassifier
xgb_clf = XGBClassifier(n_estimators = 300, learning_rate = 0.1,
                        max_depth = 5, n_jobs = -1, scale_pos_weight = weight)

xgb_clf.fit(X_train, y_train)

In [ ]:
get_clf_eval(y_test, xgb_clf.predict(X_test))

In [ ]:
# 데이터 전처리 v2
# Amount 칼럼 값의 분포 확인

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize = (5, 3))
sns.distplot(card_df.Amount)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit_transform(card_df.Amount.values.reshape(-1, 1))
# values : ndarray 형태로 출력됨. 따라서 reshape(-1, 1) 해줌

In [ ]:
# 데이터 전처리 v3
# Amount 컬럼에 StandardScaler 대신 log-scale 변경
def get_preprocessed_df(df) :
    df2 = df.copy()
    ## df2에 전처리 수행
    df2.drop('Time', axis = 1, inplace = True)
    # Amount 컬럼을 log-scale로 변경
    df2.Amount = np.log1p(df2.Amount)
    return2

In [ ]:
# 데이터 전처리 #4. 이상치 제거
# 1) 어떤 컬럼의 이상치를 제거할지 결정
# 우리가 예측하고자하는 Class와 상관관계가 높은 컬럼 선별

plt.figure(figsize = (6,6))
corr = card_df.corr()
sns.heatmap(corr, cmap = 'RdBu')
# 색깔이 진할수록 상관관계가 높음
# V14 컬럼의 이상치를 제거하기로 결정

In [ ]:
# 아니면 그냥 corr.Class_values()로 상관관계가 높은 순대로 볼 수도 있음

In [ ]:
# 그냥 V14 먼저 하기로함
# 이상치 데이터를 찾는 함수를 구현
# 평균의 범위 25% 밖의 값들을 제거하기로 결정(하위 25% 이하, 상위 75% 이상)
# 하위 25% 지점을 a, 상위 75% 지점을 b라 할 때, (b-a)w = c (w는 가중치, c는 임의의 값)을 계산한 뒤,
# 각각의 지점에서 해당 값만큼 떨어진 범위의 밖에 위치한 데이터 제거

def get_outlier(df = None, column = None, weight = 1.5) :
    # 카드 도용(사기, fraud)에 해당하는 데이터만 고려
    fraud = df[df.Class == 1][column]
    # 25% 분위와 75% 분위값을 구하기
    # 분위값 : quantile
    q25 = fraud.quantile(0.25)
    q75 = fraud.quantile(0.75)
    iqr = q75 - q25
    iqr_weight = iqr * weight
    # 이상치 데이터를 판단하는 기준값 찾기(최소기준값, 최대기준값)
    lowest = q25 - iqr_weight
    highest = q75 + iqr_weight
    # lowest보다 작거나, highest보다 큰 데이터 ==> 이상치
    outlier_index = fraud[(fraud < lowest) | (fraud > highest)].index
    return outlier_index

In [ ]:
# 이상치 찾기
get_outlier(card_df, 'V14', 1.5)

In [ ]:
def get_preprocessed_df(df) :
    df2 = df.copy()
    ## df2에 전처리 수행
    df2.drop('Time', axis = 1, inplace = True)
    # Amount 컬럼을 log-scale로 변경
    df2.Amount = np.log1p(df2.Amount)
    # V14 컬럼의 이상치 인덱스 확인
    outlier_index = get_outlier(df2, 'V14', 1.5)
    # 이상치가 있는 투플 제거
    df2.drop(outlier_index, axis = 0, inplace = True)
    return2

In [ ]:
# 데이터 전처리 v5 - 샘플링 for 데이터 편중 해결
card_df.Class.value_counts()

# 1) 다운샘플링(언더샘플링) : 다수 클래스의 개수를 소수 클래스의 개수로 맞춰주는 샘플링('0' 클래스도 '1' 클래스의 갯수만큼 줄이기)
# 이거는 거의 안씀. 무식한 방법

# 2) 업샘플링(오버샘플링) : 소수 클래스의 개수를 다수 클래스의 개수로 맞춰주는 샘플링('1' 클래스도 '0' 클래스의 개수만큼 늘리기)
# 가장 대표적인 알고리즘 : SMOTE(소수 클래스를 대표하는 데이터를 생산)

In [ ]:
# conda install - c conda-forge imbalanced-learn
from imblearn.over_sampling import SMOTE

# ModuleNotFoundError : No module named 'joblib' 에러가 발생하면
# conda install joblib 수행
smote = SMOTE(random_state = 0)

In [ ]:
X_train_over, y_train_over = smote.fit_sample(X_train, y_train)

In [ ]:
y_train.value_counts()

In [ ]:
Series(y_train_over).value_counts()

In [ ]:
lr_clf = LogisticRegression(solver = 'lbfgs', max_iter = 3000)
lr_clf.fit(X_train_over, y_train_over)  # 오버샘플링된 학습 데이터 활용
get_clf_eval(y_test, lr_clf.predict(X_test))

In [ ]:
# XGBoost with scale_pos_weight 학습 및 평가

# X_train_over가 현재 ndarray이기 때문에 DataFrame으로 바꿔줘야 함
X_train_over = DataFrame(X_train_over, columns = X_train.columns)

xgb_clf2 = XGBClassifier(n_estimators = 300, learning_rate = 0.1, max_depth = 5, n_jobs = -1)
xgb_clf2.fit(X_train_over, y_train_over)  # 오버샘플링된 학습 데이터 활용
get_clf_eval(y_test, xgb_clf2.predict(X_test))